# Proyecto Final: Clasificación de Sentimientos

In [ ]:
# Librerías principales
import pandas as pd
import numpy as np

from sklearn.naive_bayes import MultinomialNB
import torch
# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Preprocesamiento de texto
import re
from collections import Counter
#Identificar palabras clave por sentimientos
from nltk.corpus import stopwords
import nltk

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

# Configuracion
from transformers import AutoTokenizer, AutoModel
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
#Dividir el data set
from sklearn.model_selection import train_test_split

#Importamos las librerias para los algoritmos
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from imblearn.over_sampling import RandomOverSampler
from tqdm import tqdm

# COLORES DE MATPLOT y tamaño de las gráficas
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

SyntaxError: invalid syntax (2944170571.py, line 6)

In [ ]:
# Descargar stopwords si es necesario
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish')) 
nltk.download('wordnet')

In [ ]:
# Definir stopwords en español
stop_words = set(stopwords.words('spanish'))

# Inicializar lematizador y stemmer
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("spanish")

<h1> Análisis Exploratorio de Datos (EDA) </h1>

In [ ]:
# Cargar el dataset
dataset_path = "Textos_Dataset_Completo.csv"  # Cambia el path si es necesario
dataset = pd.read_csv(dataset_path, encoding="latin1")

In [ ]:
# Mostrar información básica
print("Información del Dataset:")
print(dataset.info())

In [ ]:
print("\nVista preliminar del Dataset:")
print(dataset.head())

In [ ]:
# Verificar valores nulos
print("\nValores nulos por columna:")
print(dataset.isnull().sum())

In [ ]:
#Eliminar valores nulos
dataset.dropna(inplace=True)

In [ ]:
#Verificar valores nulos
print("\nValores nulos por columna:")
print(dataset.isnull().sum())

In [ ]:
# Respuestas por pregunta
preguntas = [col for col in dataset.columns if col.startswith('1.') or col.startswith('2.') or col.startswith('3.')or col.startswith('4.')or col.startswith('5.')
             or col.startswith('6.') or col.startswith('7.') or col.startswith('8.') or col.startswith('9.') or col.startswith('10.')]
print("Cantidad de respuestas por pregunta:")
for pregunta in preguntas:
    print(f"{pregunta}: {dataset[pregunta].notnull().sum()} respuestas")

In [ ]:
#Categorias de las preguntas
categorias = {
    "Alegría": ["1.", "6."],
    "Tristeza": ["2."],
    "Estrés": ["3.", "9."],
    "Inquietud/Preocupación": ["4.", "5."],
    "Miedo": ["7.", "10."],
    "Enojo": ["8."]
}

In [ ]:
# Frecuencia de categorías de sentimiento
frecuencias = {}
for categoria, preguntas_cat in categorias.items():
    # Filtrar columnas que coincidan con los prefijos dados
    columnas = [col for col in dataset.columns if any(col.startswith(p) for p in preguntas_cat)]
    # Contar respuestas no nulas en estas columnas
    total_respuestas = dataset[columnas].notnull().sum().sum()
    frecuencias[categoria] = total_respuestas

In [ ]:
print("\nFrecuencia de respuestas por categoría de sentimiento:")
print(frecuencias)

In [ ]:
# Visualización de frecuencias corregidas
sns.barplot(x=list(frecuencias.keys()), y=list(frecuencias.values()))
plt.title("Distribución de Sentimientos en el Dataset")
plt.xlabel("Categoría de Sentimiento")
plt.ylabel("Número de Respuestas")
plt.xticks(rotation=45)
plt.show()

In [ ]:
for categoria, frecuencia in frecuencias.items():
    print(f"{categoria}: {frecuencia}")

In [ ]:
def generar_nube_palabras(data, columnas, titulo):
    # Concatenar respuestas en una sola cadena, asegurando que todo sea texto
    texto = " ".join(data[columnas].fillna("").astype(str).sum(axis=1))
    
    # Crear la nube de palabras
    nube = WordCloud(width=800, height=400, background_color="white", colormap="viridis").generate(texto)
    
    # Mostrar la nube de palabras
    plt.figure(figsize=(10, 6))
    plt.imshow(nube, interpolation="bilinear")
    plt.title(titulo, fontsize=16)
    plt.axis("off")
    plt.show()

In [ ]:
# Generar nube de palabras para cada sentimiento
for sentimiento, preguntas_cat in categorias.items():
    columnas = [col for col in dataset.columns if any(col.startswith(p) for p in preguntas_cat)]
    generar_nube_palabras(dataset, columnas, f"Nube de Palabras - {sentimiento}")

In [ ]:
def palabras_frecuentes(data, columnas, titulo, top_n=10):
    """
    Identificar las palabras más frecuentes en un conjunto de textos.
    """
    # Concatenar todas las respuestas en una sola cadena
    texto = " ".join(data[columnas].fillna("").astype(str).sum(axis=1))
    
    # Preprocesar el texto: convertir a minúsculas y eliminar caracteres especiales
    texto_limpio = re.sub(r"[^a-zA-ZáéíóúñÑ\s]", "", texto.lower())
    
    # Tokenizar el texto
    palabras = texto_limpio.split()
    
    # Filtrar palabras vacías (stopwords)
    palabras = [palabra for palabra in palabras if palabra not in stop_words]
    
    # Contar la frecuencia de cada palabra
    conteo = Counter(palabras)
    
    # Obtener las palabras más comunes
    palabras_comunes = conteo.most_common(top_n)
    
    # Mostrar resultados
    print(f"Palabras más frecuentes para {titulo}:")
    for palabra, frecuencia in palabras_comunes:
        print(f"{palabra}: {frecuencia}")
    
    # Visualizar con una gráfica de barras
    palabras, frecuencias = zip(*palabras_comunes)
    plt.bar(palabras, frecuencias)
    plt.title(f"Palabras más frecuentes - {titulo}")
    plt.xlabel("Palabras")
    plt.ylabel("Frecuencia")
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
# Identificar palabras clave para cada sentimiento
for sentimiento, preguntas_cat in categorias.items():
    columnas = [col for col in dataset.columns if any(col.startswith(p) for p in preguntas_cat)]
    palabras_frecuentes(dataset, columnas, sentimiento, top_n=10)

<h1> Preprocesamiento de Texto </h1>

In [ ]:
print(dataset.columns)

In [ ]:
def limpiar_texto(texto):
    # Convertir a minúsculas
    texto = texto.lower()
    # Eliminar caracteres especiales y números
    texto = re.sub(r"[^a-záéíóúñ\s]", "", texto)
    # Eliminar números
    texto = re.sub(r"\d+", "", texto)
    return texto

In [ ]:
def procesar_texto(texto, stop_words, lematizar=True):
    texto_limpio = limpiar_texto(texto)
    palabras = texto_limpio.split()  # Tokenización
    palabras = [palabra for palabra in palabras if palabra not in stop_words]  # Eliminar stopwords
    
    # Lematización o stemming
    if lematizar:
        palabras = [lemmatizer.lemmatize(palabra) for palabra in palabras]
    else:
        palabras = [stemmer.stem(palabra) for palabra in palabras]
    return " ".join(palabras)  # Reunir palabras en un solo string

In [ ]:
# Definir la función de procesamiento general
def procesar_todas_las_columnas(dataset, stop_words):
    """
    Aplica procesamiento a todas las columnas del dataset.
    - Limpia texto para columnas de texto.
    - Convierte valores numéricos o categóricos en cadenas para consistencia.
    """
    for col in dataset.columns:
        dataset[col] = dataset[col].fillna("").apply(
            lambda x: procesar_texto(str(x), stop_words) if isinstance(x, str) else str(x)
        )
    return dataset

# Aplicar la función al dataset completo
dataset = procesar_todas_las_columnas(dataset, stop_words)

# Verificar los resultados
print(dataset.head())

In [ ]:
dataset = procesar_todas_las_columnas(dataset, stop_words)

In [ ]:
print("\nVista preliminar del Dataset:")
dataset.head()

<h1>Categorias y frecuencia</h1>

In [ ]:
# Crear un diccionario para almacenar las respuestas por categoría
respuestas_por_categoria = {}

for categoria, preguntas_cat in categorias.items():
    # Filtrar columnas asociadas a la categoría
    columnas = [col for col in dataset.columns if any(col.startswith(p) for p in preguntas_cat)]
    # Combinar todas las respuestas de estas columnas
    respuestas_por_categoria[categoria] = dataset[columnas].fillna("").apply(lambda x: " ".join(x), axis=1)

# Convertir el diccionario a un DataFrame para inspección (opcional)
respuestas_df = pd.DataFrame(respuestas_por_categoria)

In [ ]:
# Verificar las primeras filas del DataFrame
print(respuestas_df.head())

In [ ]:
# Combinar todos los textos del dataset en una lista para la vectorización
textos = dataset.apply(lambda row: " ".join(row.astype(str)), axis=1)

# Configurar el vectorizador TF-IDF
vectorizer = TfidfVectorizer(max_features=500)  # Limitar a 500 características principales
tfidf_matrix = vectorizer.fit_transform(textos)

# Convertir a DataFrame para inspección
import pandas as pd
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Verificar el resultado
print(tfidf_df.head())

In [ ]:
# Inspeccionar las palabras más relevantes en el primer documento
palabras_importantes = tfidf_df.iloc[0].sort_values(ascending=False).head(10)
print(palabras_importantes)

In [ ]:
print(f"Total de características seleccionadas: {tfidf_df.shape[1]}")

In [ ]:
print(tfidf_matrix.shape)
print(tfidf_matrix.nnz)  # Número de elementos no nulos

In [ ]:
# Crear un mapa de calor para una muestra (por ejemplo, las primeras 50 palabras)
plt.figure(figsize=(15, 10))
sns.heatmap(tfidf_df.iloc[:50, :].T, cmap="YlGnBu", cbar=True, xticklabels=False, yticklabels=vectorizer.get_feature_names_out())
plt.title("Mapa de Calor de la Matriz TF-IDF")
plt.xlabel("Documentos")
plt.ylabel("Palabras")
plt.show()

In [ ]:
# Combinar todas las columnas relacionadas con las preguntas
dataset['Texto_Procesado'] = dataset[
    [col for col in dataset.columns if col.startswith(tuple(str(i) for i in range(1, 11)))]
].apply(lambda x: " ".join(x), axis=1)

# Vectorizar el texto procesado usando TF-IDF
vectorizer = TfidfVectorizer(max_features=500)  # Puedes ajustar el número máximo de características
tfidf_matrix = vectorizer.fit_transform(dataset['Texto_Procesado'])

# Convertir a DataFrame para inspección
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
print(tfidf_df.head())

In [ ]:
# Seleccionar las 20 palabras más frecuentes (mayores valores TF-IDF promedio)
top_words = tfidf_df.mean().sort_values(ascending=False).head(20).index

# Filtrar la matriz TF-IDF para estas palabras
tfidf_top = tfidf_df[top_words]

# Visualizar con un mapa de calor
plt.figure(figsize=(12, 8))
sns.heatmap(tfidf_top.T, cmap="viridis", annot=False, cbar=True, yticklabels=top_words)
plt.title("Mapa de Calor de la Matriz TF-IDF (Top 20 Palabras)")
plt.xlabel("Fila")
plt.ylabel("Palabras")
plt.show()

<h1> Generacion de Embeddings usando Transformers </h1>

In [ ]:
# Configuración del modelo
MODEL_NAME = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar modelo y tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

# Función para generar embeddings en lotes
def generar_embeddings_batch(textos, batch_size=16):
    embeddings = []
    with torch.no_grad():
        for i in tqdm(range(0, len(textos), batch_size)):
            batch_textos = textos[i:i + batch_size]
            inputs = tokenizer(batch_textos, return_tensors="pt", padding=True, truncation=True, max_length=512)
            inputs = {key: val.to(device) for key, val in inputs.items()}  # Mover inputs a GPU
            outputs = model(**inputs)
            batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # CLS token
            embeddings.extend(batch_embeddings)
    return np.array(embeddings)

# Generar embeddings para los textos procesados
textos = dataset['Texto_Procesado'].fillna("").tolist()
embeddings = generar_embeddings_batch(textos, batch_size=16)

# Convertir a DataFrame para inspección (opcional)
embeddings_df = pd.DataFrame(embeddings)
print(embeddings_df.head())

# Guardar los embeddings en el dataset
dataset["Embeddings"] = list(embeddings)

# Guardar los embeddings en un archivo
np.save("embeddings.npy", embeddings)

In [ ]:
#Ahora obtenemos los embeddings guardados
print(dataset["Embeddings"].head())

In [ ]:
# Reducir la dimensionalidad a 2D con PCA
pca = PCA(n_components=2)
embeddings_pca = pca.fit_transform(embeddings)

# Visualizar los embeddings
plt.figure(figsize=(10, 8))
plt.scatter(embeddings_pca[:, 0], embeddings_pca[:, 1], c='blue', alpha=0.5)
plt.title("Visualización de Embeddings (PCA)")
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.show()

<h1> Division de DataSet </h1>

In [ ]:
print("Distribución de clases en la columna 'Sentimientos':")
print(respuestas_df)

In [ ]:
# Seleccionar características (X) y etiquetas (y)
X = embeddings  # Usaremos los embeddings generados como características
y = respuestas_df  # Las etiquetas de sentimiento

In [ ]:
print(X)

In [ ]:
print(y)

In [ ]:
# División del dataset (80% entrenamiento, 20% prueba) manteniendo la proporción de clases
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Verificar la distribución de clases
print("Distribución de clases en el conjunto completo:")
print(y.value_counts())

print("\nDistribución de clases en el conjunto de entrenamiento:")
print(y_train.value_counts())

print("\nDistribución de clases en el conjunto de prueba:")
print(y_test.value_counts())

In [ ]:
# Realizar una nueva división estratificada
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Verificar nuevamente la distribución
print("Distribución de clases después de la nueva división:")
print(y_train.value_counts())

<h1>Modelos</h1>

In [ ]:
# Modelo KNN 
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
print("KNN")
print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print("Classification Report:\n", classification_report(y_test, y_pred_knn))

# Naive Bayes
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
print("\nNaive Bayes")
print("Accuracy:", accuracy_score(y_test, y_pred_nb))
print("Classification Report:\n", classification_report(y_test, y_pred_nb))

id3 = DecisionTreeClassifier(criterion='entropy', random_state=42)
id3.fit(X_train, y_train)
y_pred_id3 = id3.predict(X_test)
print("\nID3")
print("Accuracy:", accuracy_score(y_test, y_pred_id3))
print("Classification Report:\n", classification_report(y_test, y_pred_id3))
print("Tree Structure:\n", export_text(id3, feature_names=list(X.columns)))

c45 = DecisionTreeClassifier(criterion='gini', random_state=42)
c45.fit(X_train, y_train)
y_pred_c45 = c45.predict(X_test)
print("\nC4.5")
print("Accuracy:", accuracy_score(y_test, y_pred_c45))
print("Classification Report:\n", classification_report(y_test, y_pred_c45))
print("Tree Structure:\n", export_text(c45, feature_names=list(X.columns)))

In [ ]:
# Aplicar sobremuestreo
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

# Verificar la distribución después del sobremuestreo
print("Distribución de clases después del sobremuestreo:")
print(pd.Series(y_train_resampled).value_counts())

In [ ]:
# Probar SVM con kernel lineal
svm_linear = SVC(kernel="linear", C=1, random_state=42)
svm_linear.fit(X_train, y_train)

# Predicciones en el conjunto de prueba
y_pred_svm_linear = svm_linear.predict(X_test)

# Evaluar el rendimiento
print("\n--- Resultados para SVM (Kernel Lineal) ---")
print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred_svm_linear))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_svm_linear))
print(f"Precisión: {accuracy_score(y_test, y_pred_svm_linear):.2f}")

# Probar SVM con kernel RBF
svm_rbf = SVC(kernel="rbf", C=1, gamma=0.1, random_state=42)
svm_rbf.fit(X_train, y_train)

# Predicciones en el conjunto de prueba
y_pred_svm_rbf = svm_rbf.predict(X_test)

# Evaluar el rendimiento
print("\n--- Resultados para SVM (Kernel RBF) ---")
print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred_svm_rbf))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_svm_rbf))
print(f"Precisión: {accuracy_score(y_test, y_pred_svm_rbf):.2f}")